# Use case proposed in [issue #78](https://github.com/bsorrentino/langgraph4j/issues/78) by [ikwattro](https://github.com/ikwattro)

## How to Reset memory thread 

In [1]:
String userHomeDir = System.getProperty("user.home");
String localRespoUrl = "file://" + userHomeDir + "/.m2/repository/";
String langchain4jVersion = "1.0.0-beta1";
String langgraph4jVersion = "1.4-SNAPSHOT";

In [2]:
%dependency /add-repo local \{localRespoUrl} release|never snapshot|always
// %dependency /list-repos
%dependency /add org.slf4j:slf4j-jdk14:2.0.9
%dependency /add org.bsc.langgraph4j:langgraph4j-core:\{langgraph4jVersion}
%dependency /add org.bsc.langgraph4j:langgraph4j-langchain4j:\{langgraph4jVersion}
%dependency /add org.bsc.langgraph4j:langgraph4j-agent-executor:\{langgraph4jVersion}
%dependency /add dev.langchain4j:langchain4j:\{langchain4jVersion}
%dependency /add dev.langchain4j:langchain4j-ollama:\{langchain4jVersion}

%dependency /resolve

Repository local url: file:///Users/bsorrentino/.m2/repository/ added.
Adding dependency org.slf4j:slf4j-jdk14:2.0.9
Adding dependency org.bsc.langgraph4j:langgraph4j-core:1.3-SNAPSHOT
Adding dependency org.bsc.langgraph4j:langgraph4j-langchain4j:1.3-SNAPSHOT
Adding dependency org.bsc.langgraph4j:langgraph4j-agent-executor:1.3-SNAPSHOT
Adding dependency dev.langchain4j:langchain4j:0.36.2
Adding dependency dev.langchain4j:langchain4j-ollama:0.36.2
Solving dependencies
Resolved artifacts count: 65
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/slf4j/slf4j-jdk14/2.0.9/slf4j-jdk14-2.0.9.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/slf4j/slf4j-api/2.0.9/slf4j-api-2.0.9.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/bsc/langgraph4j/langgraph4j-core/1.3-SNAPSHOT/langgraph4j-core-1.3-SNAPSHOT.jar
Add to classpath: /Users/bsorrentino/

**Initialize Logger**

In [3]:
try( var file = new java.io.FileInputStream("./logging.properties")) {
    var lm = java.util.logging.LogManager.getLogManager();
    lm.checkAccess(); 
    lm.readConfiguration( file );
}

var log = org.slf4j.LoggerFactory.getLogger("AdaptiveRag");


In [37]:
import org.bsc.langgraph4j.state.AgentState;
import org.bsc.langgraph4j.state.Channel;
import java.util.Optional;

public class State extends AgentState {
    static Map<String, Channel<?>> SCHEMA = Map.of();
    
    public State(Map<String,Object> initData) {
        super(initData);
    }
}

In [38]:
import dev.langchain4j.model.ollama.OllamaChatModel;
import dev.langchain4j.model.chat.Capability;

var chatLanguageModel = OllamaChatModel.builder()
            .modelName("llama3.2")
            .baseUrl("http://localhost:11434")
            //.supportedCapabilities( Capability.RESPONSE_FORMAT_JSON_SCHEMA )
            .temperature(0.0d)
            .logRequests(true)
            .logResponses(true)
            .build();


In [54]:
import org.bsc.langgraph4j.action.NodeActionWithConfig;
import dev.langchain4j.service.AiServices;
import dev.langchain4j.model.chat.ChatLanguageModel;
import dev.langchain4j.memory.chat.MessageWindowChatMemory;
import dev.langchain4j.service.MemoryId;
import dev.langchain4j.service.UserMessage;

class TestAction implements NodeActionWithConfig<State> {

    private final TestAssistant testAssistant;

    public TestAction(ChatLanguageModel chatLanguageModel) {
        this.testAssistant = AiServices.builder(TestAssistant.class)
                .chatLanguageModel(chatLanguageModel)
                .chatMemoryProvider(memoryId -> MessageWindowChatMemory.withMaxMessages(20))
                .build();
    }

    @Override
    public Map<String, Object> apply(State state, RunnableConfig runnableConfig) throws Exception {
        var conversation = state.<String>value("conversation")
                                .orElseThrow(() -> new IllegalStateException("No conversation found in state"));

        var threadId = runnableConfig.threadId().orElse( "unknown" );

        return Map.of("response", testAssistant.test( threadId, conversation));
    }

    public interface TestAssistant {
        String test(@MemoryId String memoryId, @UserMessage String query);
    }
}


In [55]:
import org.bsc.langgraph4j.StateGraph;
import static org.bsc.langgraph4j.action.AsyncNodeActionWithConfig.node_async;
import static org.bsc.langgraph4j.StateGraph.START;
import static org.bsc.langgraph4j.StateGraph.END;

var testAction = new TestAction(chatLanguageModel);

var workflow =  new StateGraph<>(State.SCHEMA, State::new)
            .addNode("agent", node_async(testAction))
            .addEdge(START, "agent")
            .addEdge("agent", END);

In [57]:
import org.bsc.langgraph4j.checkpoint.MemorySaver;
import org.bsc.langgraph4j.CompileConfig;
import org.bsc.langgraph4j.RunnableConfig;

var memory = new MemorySaver();
var compileConfig = CompileConfig.builder()
        .checkpointSaver(memory)
        .build();

var runnableConfig1 = RunnableConfig.builder()
        .threadId("conversation-num-1")
        .build();
var runnableConfig2 = RunnableConfig.builder()
        .threadId("conversation-num-2")
        .build();

var app = workflow.compile(compileConfig);

app.invoke( Map.of("conversation", "Hi, my name is Chris"), runnableConfig1 )
        .map( r -> r.data() )
        .ifPresent( System.out::println  );

app.invoke( Map.of("conversation", "what's my name ?"), runnableConfig1 )
        .map( r -> r.data() )
        .ifPresent( System.out::println );

app.invoke( Map.of("conversation", "What's my name ?"), runnableConfig2 )
        .map( r -> r.data() )
        .ifPresent( System.out::println );

    

START 


{response=I think we've established that by now! But yes, your name is indeed Chris. How can I assist you today?, conversation=Hi, my name is Chris}


START 


{response=Déjà vu again! You're right, we've been over this a few times already. Your name is still Chris. Is there something specific you'd like to talk about or ask for help with?, conversation=what's my name ?}


START 


{response=I don't know your name. I'm a large language model, I don't have the ability to store or recall information about individual users. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations.

If you'd like to share your name with me, I'd be happy to chat with you!, conversation=What's my name ?}
